In [1]:
import sys
sys.path.append("C:/Users/Danny Farone/Documents/GitHub/Data-Science-Job-Salaries/src/")
from required_libraries import *

In [2]:
df = pd.read_csv(r'C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\external\ds_salaries_external.csv', header = 0)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Analyst,98000,USD,98000,US,0,US,M
1,2021,EX,FT,Data Analyst,150000,USD,150000,IN,100,US,L
2,2021,MI,FT,Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,FT,Data Analyst,11000000,HUF,36259,HU,50,US,L
4,2021,EN,FT,Data Analyst,55000,USD,55000,US,50,US,S
...,...,...,...,...,...,...,...,...,...,...,...
601,2021,MI,FT,Machine Learning Engineer,81000,EUR,95746,DE,100,US,S
602,2022,EN,FT,Machine Learning Engineer,150000,USD,150000,AU,100,AU,S
603,2022,MI,FT,Machine Learning Engineer,50000,EUR,54957,GR,0,GR,M
604,2022,MI,FT,Machine Learning Engineer,50000,EUR,54957,GR,0,GR,M


In [3]:
df = df.drop(['salary'], axis = 1)
df

,work_year,experience_level,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Analyst,USD,98000,US,0,US,M
1,2021,EX,FT,Data Analyst,USD,150000,IN,100,US,L
2,2021,MI,FT,Data Analyst,USD,100000,US,100,US,M
3,2021,MI,FT,Data Analyst,HUF,36259,HU,50,US,L
4,2021,EN,FT,Data Analyst,USD,55000,US,50,US,S
...,...,...,...,...,...,...,...,...,...,...
601,2021,MI,FT,Machine Learning Engineer,EUR,95746,DE,100,US,S
602,2022,EN,FT,Machine Learning Engineer,USD,150000,AU,100,AU,S
603,2022,MI,FT,Machine Learning Engineer,EUR,54957,GR,0,GR,M
604,2022,MI,FT,Machine Learning Engineer,EUR,54957,GR,0,GR,M


In [4]:
#look at unique values for the categorical variables
df.nunique()

work_year               3
experience_level        4
employment_type         4
job_title               4
salary_currency        17
salary_in_usd         368
employee_residence     57
remote_ratio            3
company_location       50
company_size            3
dtype: int64

In [5]:
df['experience_level'].value_counts()

SE    280
MI    212
EN     88
EX     26
Name: experience_level, dtype: int64

In [6]:
df_experience_level = pd.DataFrame(df['experience_level'].value_counts().reset_index())

experience_level_dummies = pd.get_dummies(df['experience_level'])
#Concat the dummy variables to the main dataset
df = pd.concat([df,experience_level_dummies], axis =1)
#and drop the original variable from the data
df = df.drop('experience_level', axis = 1)
experience_level_dummies.value_counts()

EN  EX  MI  SE
0   0   0   1     280
        1   0     212
1   0   0   0      88
0   1   0   0      26
dtype: int64

In [7]:
df = df.rename(columns={'EN' : 'Entry_Exp', 'EX' : 'Expert_Exp', 'MI' : 'Mid_Exp', 'SE' : ' Senior_Exp'})

In [8]:
df

,work_year,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp
0,2020,FT,Data Analyst,USD,98000,US,0,US,M,0,0,1,0
1,2021,FT,Data Analyst,USD,150000,IN,100,US,L,0,1,0,0
2,2021,FT,Data Analyst,USD,100000,US,100,US,M,0,0,1,0
3,2021,FT,Data Analyst,HUF,36259,HU,50,US,L,0,0,1,0
4,2021,FT,Data Analyst,USD,55000,US,50,US,S,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,FT,Machine Learning Engineer,EUR,95746,DE,100,US,S,0,0,1,0
602,2022,FT,Machine Learning Engineer,USD,150000,AU,100,AU,S,1,0,0,0
603,2022,FT,Machine Learning Engineer,EUR,54957,GR,0,GR,M,0,0,1,0
604,2022,FT,Machine Learning Engineer,EUR,54957,GR,0,GR,M,0,0,1,0


In [9]:
df['employment_type'].value_counts()

FT    588
PT      9
CT      5
FL      4
Name: employment_type, dtype: int64

In [10]:
for i in range(len(df['employment_type'])):
    if df['employment_type'][i] == 'FT':
        df['employment_type'][i] = 1
    else:
        df['employment_type'][i] = 0

In [11]:
df['employment_type'].value_counts()

1    588
0     18
Name: employment_type, dtype: int64

In [12]:
df

,work_year,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp
0,2020,1,Data Analyst,USD,98000,US,0,US,M,0,0,1,0
1,2021,1,Data Analyst,USD,150000,IN,100,US,L,0,1,0,0
2,2021,1,Data Analyst,USD,100000,US,100,US,M,0,0,1,0
3,2021,1,Data Analyst,HUF,36259,HU,50,US,L,0,0,1,0
4,2021,1,Data Analyst,USD,55000,US,50,US,S,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,Machine Learning Engineer,EUR,95746,DE,100,US,S,0,0,1,0
602,2022,1,Machine Learning Engineer,USD,150000,AU,100,AU,S,1,0,0,0
603,2022,1,Machine Learning Engineer,EUR,54957,GR,0,GR,M,0,0,1,0
604,2022,1,Machine Learning Engineer,EUR,54957,GR,0,GR,M,0,0,1,0


In [13]:
df['job_title'].value_counts()

Data Scientist               218
Data Engineer                177
Data Analyst                 131
Machine Learning Engineer     80
Name: job_title, dtype: int64

In [14]:
job_title = pd.DataFrame(df['job_title'].value_counts().reset_index())

job_title_dummies = pd.get_dummies(df['job_title'])
#Concat the dummy variables to the main dataset 
df = pd.concat([df, job_title_dummies], axis = 1)
#and drop the original variable from the data
df = df.drop('job_title', axis = 1)
job_title_dummies.value_counts()

Data Analyst  Data Engineer  Data Scientist  Machine Learning Engineer
0             0              1               0                            218
              1              0               0                            177
1             0              0               0                            131
0             0              0               1                             80
dtype: int64

In [15]:
df

,work_year,employment_type,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer
0,2020,1,USD,98000,US,0,US,M,0,0,1,0,1,0,0,0
1,2021,1,USD,150000,IN,100,US,L,0,1,0,0,1,0,0,0
2,2021,1,USD,100000,US,100,US,M,0,0,1,0,1,0,0,0
3,2021,1,HUF,36259,HU,50,US,L,0,0,1,0,1,0,0,0
4,2021,1,USD,55000,US,50,US,S,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,EUR,95746,DE,100,US,S,0,0,1,0,0,0,0,1
602,2022,1,USD,150000,AU,100,AU,S,1,0,0,0,0,0,0,1
603,2022,1,EUR,54957,GR,0,GR,M,0,0,1,0,0,0,0,1
604,2022,1,EUR,54957,GR,0,GR,M,0,0,1,0,0,0,0,1


In [16]:
df['salary_currency'].value_counts()

USD    398
EUR     95
GBP     44
INR     26
CAD     18
JPY      3
PLN      3
TRY      3
DKK      2
AUD      2
BRL      2
MXN      2
HUF      2
SGD      2
CNY      2
CLP      1
CHF      1
Name: salary_currency, dtype: int64

In [17]:
USD = []
EUR = []
GBP = []
INR = []
Other_Currency = []

In [18]:
for currency in df['salary_currency']:
    if currency == 'USD':
        USD.append(1)
        EUR.append(0)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'EUR':
        USD.append(0)
        EUR.append(1)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'GBP':
        USD.append(0)
        EUR.append(0)
        GBP.append(1)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'INR':
        USD.append(0)
        EUR.append(0)
        GBP.append(0)
        INR.append(1)
        Other_Currency.append(0)
    else:
        USD.append(0)
        EUR.append(0)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(1)

In [19]:
df['USD'] = USD
df['EUR'] = EUR
df['GBP'] = GBP
df['INR'] = INR
df['Other_Currency'] = Other_Currency

In [20]:
df = df.drop('salary_currency',axis = 1)

In [21]:
df

,work_year,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer,USD,EUR,GBP,INR,Other_Currency
0,2020,1,98000,US,0,US,M,0,0,1,0,1,0,0,0,1,0,0,0,0
1,2021,1,150000,IN,100,US,L,0,1,0,0,1,0,0,0,1,0,0,0,0
2,2021,1,100000,US,100,US,M,0,0,1,0,1,0,0,0,1,0,0,0,0
3,2021,1,36259,HU,50,US,L,0,0,1,0,1,0,0,0,0,0,0,0,1
4,2021,1,55000,US,50,US,S,1,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,95746,DE,100,US,S,0,0,1,0,0,0,0,1,0,1,0,0,0
602,2022,1,150000,AU,100,AU,S,1,0,0,0,0,0,0,1,1,0,0,0,0
603,2022,1,54957,GR,0,GR,M,0,0,1,0,0,0,0,1,0,1,0,0,0
604,2022,1,54957,GR,0,GR,M,0,0,1,0,0,0,0,1,0,1,0,0,0


In [22]:
df['employee_residence'].value_counts()

US    332
GB     44
IN     29
CA     29
DE     25
     ... 
HN      1
BG      1
LU      1
KE      1
IE      1
Name: employee_residence, Length: 57, dtype: int64

In [23]:
for i in range(len(df['employee_residence'])):
    if df['employee_residence'][i] == 'US':
        df['employee_residence'][i] = 1
    else:
        df['employee_residence'][i] = 0

In [24]:
df = df.rename(columns={'employment_type' : 'full_time_employee', 'employee_residence' : 'US_resident'})

In [25]:
df

,work_year,full_time_employee,salary_in_usd,US_resident,remote_ratio,company_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer,USD,EUR,GBP,INR,Other_Currency
0,2020,1,98000,1,0,US,M,0,0,1,0,1,0,0,0,1,0,0,0,0
1,2021,1,150000,0,100,US,L,0,1,0,0,1,0,0,0,1,0,0,0,0
2,2021,1,100000,1,100,US,M,0,0,1,0,1,0,0,0,1,0,0,0,0
3,2021,1,36259,0,50,US,L,0,0,1,0,1,0,0,0,0,0,0,0,1
4,2021,1,55000,1,50,US,S,1,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,95746,0,100,US,S,0,0,1,0,0,0,0,1,0,1,0,0,0
602,2022,1,150000,0,100,AU,S,1,0,0,0,0,0,0,1,1,0,0,0,0
603,2022,1,54957,0,0,GR,M,0,0,1,0,0,0,0,1,0,1,0,0,0
604,2022,1,54957,0,0,GR,M,0,0,1,0,0,0,0,1,0,1,0,0,0


In [26]:
df['company_location'].value_counts()

US    355
GB     47
CA     30
DE     28
IN     23
FR     15
ES     14
GR     11
JP      6
PT      4
NL      4
PL      4
AT      4
AE      3
AU      3
BR      3
MX      3
TR      3
LU      3
PK      3
DK      3
NG      2
RU      2
SI      2
IT      2
CH      2
CN      2
CZ      2
BE      2
MY      1
EE      1
IQ      1
HR      1
CO      1
SG      1
VN      1
DZ      1
CL      1
IL      1
UA      1
HU      1
AS      1
NZ      1
KE      1
IR      1
MT      1
HN      1
RO      1
MD      1
IE      1
Name: company_location, dtype: int64

In [27]:
for i in range(len(df['company_location'])):
    if df['company_location'][i] == 'US':
        df['company_location'][i] = 1
    else:
        df['company_location'][i] = 0

In [28]:
df = df.rename(columns={'company_location' : 'US_location'})

In [29]:
df

,work_year,full_time_employee,salary_in_usd,US_resident,remote_ratio,US_location,company_size,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer,USD,EUR,GBP,INR,Other_Currency
0,2020,1,98000,1,0,1,M,0,0,1,0,1,0,0,0,1,0,0,0,0
1,2021,1,150000,0,100,1,L,0,1,0,0,1,0,0,0,1,0,0,0,0
2,2021,1,100000,1,100,1,M,0,0,1,0,1,0,0,0,1,0,0,0,0
3,2021,1,36259,0,50,1,L,0,0,1,0,1,0,0,0,0,0,0,0,1
4,2021,1,55000,1,50,1,S,1,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,95746,0,100,1,S,0,0,1,0,0,0,0,1,0,1,0,0,0
602,2022,1,150000,0,100,0,S,1,0,0,0,0,0,0,1,1,0,0,0,0
603,2022,1,54957,0,0,0,M,0,0,1,0,0,0,0,1,0,1,0,0,0
604,2022,1,54957,0,0,0,M,0,0,1,0,0,0,0,1,0,1,0,0,0


In [30]:
df['company_size'].value_counts()

M    325
L    198
S     83
Name: company_size, dtype: int64

In [31]:
company_size = pd.DataFrame(df['company_size'].value_counts().reset_index())

company_size_dummies = pd.get_dummies(df['company_size'])
#Concat the dummy variables to the main dataset 
df = pd.concat([df, company_size_dummies], axis = 1)
#and drop the original variable from the data
df = df.drop('company_size', axis = 1)
company_size_dummies.value_counts()

L  M  S
0  1  0    325
1  0  0    198
0  0  1     83
dtype: int64

In [32]:
df = df.rename(columns={'L' : 'Large_Company', 'M' : 'Midsize_Company', 'S' : 'Small_Company'})

In [33]:
df

,work_year,full_time_employee,salary_in_usd,US_resident,remote_ratio,US_location,Entry_Exp,Expert_Exp,Mid_Exp,Senior_Exp,Data Analyst,Data Engineer,Data Scientist,Machine Learning Engineer,USD,EUR,GBP,INR,Other_Currency,Large_Company,Midsize_Company,Small_Company
0,2020,1,98000,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
1,2021,1,150000,0,100,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0
2,2021,1,100000,1,100,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
3,2021,1,36259,0,50,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0
4,2021,1,55000,1,50,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2021,1,95746,0,100,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1
602,2022,1,150000,0,100,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
603,2022,1,54957,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
604,2022,1,54957,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0


In [34]:
df.to_csv(r'C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\interim\ds_salaries_interim.csv', index=False)